<a href="https://colab.research.google.com/github/drfinkus/gpt-2-simple/blob/master/Train_GPT_2_1_5b_Text_Generating_Model_w_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Train a GPT-2-1.5b Text-Generating Model w/ GPU For Free 

by [drfinkus](https://github.com/drfinkus)

*Last updated: April 23rd, 2020*

This is a fork of the `gpt-2-simple` implementation of [Max Woolf](http://minimaxir.com). 

The objective of this fork is to leverage the new Tesla P100 GPUs available on Google Colab to enable retraining the GPT-2-1.5b model for free using Collaboratory.

The main problem with the original implementation is that it offers only two optimizers, Adam and SGD. 

Adam is **too memory intensive**, and does not fit in memory. SGD works, but it is a **poor optimizer** and training does not converge.

We solve this by implementing an [Adafactor](https://arxiv.org/abs/1804.04235) optimizer, based on the `tensor2tensor` implementation. This solution is more memory efficient than Adam and fits in memory for the P100 GPU.

To get started:

1. Copy this notebook to your Google Drive to keep it and save your changes. (File -> Save a Copy in Drive)
2. Run the cells below to clone the repository; files persist between runtime restarts so no need to re-run


In [0]:
!git clone https://github.com/drfinkus/gpt-2-simple
import os
os.chdir('/content/gpt-2-simple')

3. Run the cells below to install requirements (this needs to run at every runtime restart)

In [0]:
%tensorflow_version 1.x
!pip install -r requirements.txt
import gpt_2_simple.gpt_2 as gpt2
from datetime import datetime
from google.colab import files

## GPU

Colaboratory uses either an Nvidia T4 GPU, an Nvidia K80 GPU, or more recently a Tesla P100 GPU with 16GB VRAM. 

The Tesla P100 is the only GPU which is able to train the `1558M` extra-large model, or the `774M` large model.. The T4 may be used to train up to `355M`, or medium model.

You can verify which GPU is active by running the cell below. If you **do not have a P100 attached**, you will need to retry until you get one.

In [0]:
!nvidia-smi

## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk
* `774M`: the "large" model, can only be finetuned with a Tesla P100 GPU
* `1558M`: the "extra large", true model, can only be finetuned with a Tesla P100 GPU

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [0]:
gpt2.download_gpt2(model_name="1558M")

## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

**It is highly recommended to upgrade your Google Drive to 100GB storage, so that you can enable persistent checkpoints below.**

In [0]:
gpt2.mount_gdrive()

## Uploading a Text File to be Trained to Colaboratory

In the Colaboratory Notebook sidebar on the left of the screen, select *Files*. From there you can upload files:

![alt text](https://i.imgur.com/TGcZT4h.png)

Upload **any smaller text file**  (<10 MB) and update the file name in the cell below, then run the cell.

Note: make sure you use a full path to the filename.

In [0]:
file_name = "/content/tinyshakespeare.txt"

If your text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

In [0]:
gpt2.copy_file_from_gdrive(file_name)

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in your Google Drive by default, under `/content/drive/My Drive/checkpoint/run1`. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

If you wish to disable this, you can comment out the `checkpoint_dir` parameter below, which will save checkpoints to the Colab runtime storage only.

### Training the model

You can only use `adafactor` as an optimizer parameter to train the large and extra-large models, using a Tesla P100 GPU. You can try `sgd` as well, but performance is very poor.

When finetuning the model, it is important to keep an eye on the sample quality. Finding the right loss is an art rather than a science. Training the model too much (e.g. to a loss well below 1), especially on small datasets, can make it forget too much of its previous knowledge and significantly degrade quality.

You should see good results within 10-20 epochs, and you should keep an eye on sample quality, especially with an average loss below 2.1.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
*  **`checkpoint_dir`**: Path to the directory where you want to save your checkpoint; Google Drive, if mounted, it will be available in `/content/drive/My Drive/`
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data). Does not influence Adafactor optimizer.
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicates. 

In [0]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              checkpoint_dir='/content/drive/My Drive/checkpoint',
              model_name='1558M',
              steps=-1,
              restore_from='fresh',
              optimizer='adafactor',
              learning_rate=0.0001,
              run_name='run1',
              print_every=10,
              sample_every=100,
              save_every=500
              )

If you did not use persistent checkpoints (i.e. you saved checkpoints to runtime disk), you can now copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [0]:
gpt2.copy_checkpoint_to_gdrive(run_name='run1')

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [0]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [0]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [0]:
gpt2.generate(sess, run_name='run1')

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [0]:
gpt2.generate(sess,
              length=250,
              temperature=0.7,
              prefix="LORD",
              nsamples=5,
              batch_size=5
              )

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [0]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=500,
                      temperature=0.7,
                      nsamples=100,
                      batch_size=20
                      )

In [0]:
# may have to run twice to get file to download
files.download(gen_file)

## Generate Text From The Pretrained Model

If you want to generate text from the pretrained model, not a finetuned model, pass `model_name` to `gpt2.load_gpt2()` and `gpt2.generate()`.

This is currently the only way to generate text from the 774M or 1558M models with this notebook.

In [0]:
model_name = "1558M"

gpt2.download_gpt2(model_name=model_name)

In [0]:
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, model_name=model_name)

In [0]:
gpt2.generate(sess,
              model_name=model_name,
              prefix="The secret of life is",
              length=100,
              temperature=0.7,
              top_p=0.9,
              nsamples=5,
              batch_size=5
              )

# Force-kill

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [0]:
!kill -9 -1

# LICENSE

MIT License

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.